In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [2]:
import numba.extending

@numba.extending.overload(np.clip)
def np_clip(a, a_min, a_max, out=None):
    def np_clip_impl(a, a_min, a_max, out=None):
        if out is None:
            out = np.empty_like(a)
        for i in range(len(a)):
            if a[i] < a_min:
                out[i] = a_min
            elif a[i] > a_max:
                out[i] = a_max
            else:
                out[i] = a[i]
        return out
    return np_clip_impl

In [3]:
# some useful functions
# 
# you may need to comment out the 'numba' bits if your system can't install numba (like the gpvms...)
#

@numba.jit(nopython=True)
def costheta_numba(p1x,p1y,p1z,p1mag,
                   p2x,p2y,p2z,p2mag):
    return np.clip(np.where((p1mag>0.0)&(p2mag>0.0),
                            (p1x*p2x+p1y*p2y+p1z*p2z)/p1mag/p2mag,
                            np.nan),
                   -1.0,1.0)

def eval_costheta(df,suffix1="",suffix2=""):
    return costheta_numba(df.loc[:,"px"+suffix1].values,df.loc[:,"py"+suffix1].values,df.loc[:,"pz"+suffix1].values,df.loc[:,"p"+suffix1].values,
                          df.loc[:,"px"+suffix2].values,df.loc[:,"py"+suffix2].values,df.loc[:,"pz"+suffix2].values,df.loc[:,"p"+suffix2].values)

    
@numba.jit(nopython=True)
def q3_numba(p1x,p1y,p1z,p2x,p2y,p2z):
    return np.sqrt((p1x-p2x)**2+(p1y-p2y)**2+(p1z-p2z)**2)

def eval_q3(df,suffix1="",suffix2="_mu"):
    return q3_numba(df.loc[:,"px"+suffix1].values,df.loc[:,"py"+suffix1].values,df.loc[:,"pz"+suffix1].values,
                    df.loc[:,"px"+suffix2].values,df.loc[:,"py"+suffix2].values,df.loc[:,"pz"+suffix2].values)

In [4]:
def remove_duplicate_files(sig_files,bkg_files,bkg_path,aa_run,aa_label,ac_run,ac_label):
    for f in sig_files:
        r = int(f.split("/")[-1].split("_")[-2])
        sr = f.split("/")[-1].split("_")[-1].split(".")[0]
        run_set=""
        if(r==aa_run):
            run_set=aa_label
        if(r==ac_run):
            run_set=ac_label
    
        fname = "%s_%s/sampler_hist_%s.root"%(bkg_path,run_set,sr)
        try:
            bkg_files.remove(fname)
        except:
            print("%s not found. (%s)"%(fname,f))
            sig_files.remove(f)
    all_files = sig_files + bkg_files
    return all_files,sig_files

In [5]:
root_filenames_Set1Run1_Sigs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_SignalFiles/*.root")
root_filenames_Set1Run1_Bkgs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_A[A,C]/*.root")
root_filenames_Set1Run3_Sigs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3_SignalFiles/*.root")
root_filenames_Set1Run3_Bkgs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3b_A[B,C]/*.root")

In [6]:
root_filenames_Set1Run1,root_filenames_Set1Run1_Sigs = remove_duplicate_files(root_filenames_Set1Run1_Sigs,
                                                                              root_filenames_Set1Run1_Bkgs,
                                                                              "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1",
                                                                              6693842,"AA",6693859,"AC")
root_filenames_Set1Run3,root_filenames_Set1Run3_Sigs = remove_duplicate_files(root_filenames_Set1Run3_Sigs,
                                                                              root_filenames_Set1Run3_Bkgs,
                                                                              "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3b",
                                                                              7165574,"AB",7165592,"AC")

/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_AC/sampler_hist_-1.root not found. (/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_SignalFiles/sampler_hist_6693859_-1.root)


In [7]:
root_filenames_CV   = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/CVSet/*/*/sampler*.root")
root_filenames_Set1 = root_filenames_Set1Run1+root_filenames_Set1Run3
root_filenames_Set2 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/*/*/sampler*.root")
root_filenames_Set3 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/*/*/sampler*.root")
root_filenames_Set4 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set4/*/*/sampler*.root")

In [8]:
def convert_root(root_filenames):
    t_df = []
    p_df = []
    pot_df = []

    file_count = 0
    event_count = 0
    print("Processing %d files" % len(root_filenames))

    for root_filename in root_filenames:
    
        try:
            p_df.append(uproot.open(root_filename)['mcana/particle_tree'].pandas.df())
            t_df.append(uproot.open(root_filename)['mcana/mctruth_tree'].pandas.df())
            pot_df.append(uproot.open(root_filename)['potana/pot_tree'].pandas.df())
        except:
            print("File %s, trees not found."%root_filename)
    
        event_count += len(t_df[-1])
        file_count += 1
        if file_count%500==0:
            print("\tProcessed %d files. %d events processed." % (file_count,event_count))

    p_df = pd.concat(p_df)
    t_df = pd.concat(t_df)
    pot_df = pd.concat(pot_df)

    p_df.set_index(["run","subrun","event","truth_index","p_index"],inplace=True)
    t_df.set_index(["run","subrun","event","truth_index"],inplace=True)
    pot_df.set_index(["run","subrun"],inplace=True)
        
    print("Have dataframe objects. Total events is %d." % len(t_df))
    
    #make a ke column
    p_df["ke"] = p_df["e"]-p_df["mass"]
    
    return t_df,p_df,pot_df

In [9]:
def create_finalstate_df(p_df):
    df_n = pd.DataFrame()
    
    df_n["n_mu"] = ((p_df["status"]==1)&(p_df["pdgcode"]==13)).replace(False,np.nan)
    df_n["n_e"] = ((p_df["status"]==1)&(p_df["pdgcode"]==11)).replace(False,np.nan)
    df_n["n_p_40MeV"] = ((p_df["status"]==1)&(p_df["pdgcode"]==2212)&((p_df["e"]-p_df["mass"])>0.03)).replace(False,np.nan)
    df_n["n_pi0"] = ((p_df["status"]==1)&((p_df["pdgcode"]==111))).replace(False,np.nan)
    df_n["n_chpi"] = ((p_df["status"]==1)&((p_df["pdgcode"]==211)^(p_df["pdgcode"]==-211))).replace(False,np.nan)
    df_n["n_gamma"] = ((p_df["status"]==1)&((p_df["pdgcode"]==22))&(p_df["e"]>0.02)).replace(False,np.nan)
    df_n = df_n.groupby(["run","subrun","event","truth_index"]).agg("sum")
    
    return df_n

In [10]:
#groupings for final state protons (up to 4), pi0, gammas, and leptons. And initial neutrino.
def group_particle_df(p_df):
    p_df_p_grouped = p_df.query("status==1 and pdgcode==2212").sort_values(by=["e"],ascending=False).groupby(["run","subrun","event","truth_index"])
    p_df_p1 = p_df_p_grouped.nth(0)
    p_df_p2 = p_df_p_grouped.nth(1)
    p_df_p3 = p_df_p_grouped.nth(2)
    p_df_p4 = p_df_p_grouped.nth(3)

    p_df_pi0_grouped = p_df.query("status==1 and pdgcode==111").sort_values(by=["e"],ascending=False).groupby(["run","subrun","event","truth_index"])
    p_df_pi0 = p_df_pi0_grouped.nth(0)

    p_df_gamma_grouped = p_df.query("status==1 and pdgcode==22").sort_values(by=["e"],ascending=False).groupby(["run","subrun","event","truth_index"])
    p_df_gamma = p_df_gamma_grouped.nth(0)

    p_df_lep = p_df.query("status==1 and (pdgcode==13 or pdgcode==-13 or pdgcode==11 or pdgcode==-11 or pdgcode==12 or pdgcode==-12 or pdgcode==14 or pdgcode==-14)").groupby(["run","subrun","event","truth_index"]).first()
    p_df_nu = p_df.query("status==0 and (pdgcode==12 or pdgcode==-12 or pdgcode==14 or pdgcode==-14)").groupby(["run","subrun","event","truth_index"]).first()
    
    return p_df_nu,p_df_lep,p_df_p1,p_df_p2,p_df_p3,p_df_p4,p_df_pi0,p_df_gamma

In [11]:
def event_df_calcs(df_ev_t):
    df_ev_t["costheta_lep"] = eval_costheta(df=df_ev_t,suffix1="",suffix2="_lep")
    df_ev_t["costheta_p1"] = eval_costheta(df=df_ev_t,suffix1="",suffix2="_p1")
    
    return df_ev_t

In [12]:
def create_event_df(t_df,p_df):
    df_n = create_finalstate_df(p_df)
    p_df_nu,p_df_lep,p_df_p1,p_df_p2,p_df_p3,p_df_p4,p_df_pi0,p_df_gamma = group_particle_df(p_df)
    
    df_ev_t = t_df.copy()
    df_ev_t = df_ev_t.merge(p_df_nu,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_nu"])
    df_ev_t = df_ev_t.merge(p_df_lep,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_lep"])
    df_ev_t = df_ev_t.merge(p_df_p1,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_p1"])
    df_ev_t = df_ev_t.merge(p_df_p2,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_p2"])
    df_ev_t = df_ev_t.merge(p_df_p3,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_p3"])
    df_ev_t = df_ev_t.merge(p_df_p3,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_p4"])
    df_ev_t = df_ev_t.merge(p_df_pi0,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_pi0"])
    df_ev_t = df_ev_t.merge(p_df_gamma,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_gamma"])
    df_ev_t = df_ev_t.merge(df_n,how="left",on=["run","subrun","event","truth_index"])
    
    df_ev_t = event_df_calcs(df_ev_t)
    
    return df_ev_t

In [13]:
def write_hdf(name,t_df,p_df,pot_df,df_ev_t):
    t_df.to_hdf(name,"t_df")
    p_df.to_hdf(name,"p_df")
    pot_df.to_hdf(name,"pot_df")
    df_ev_t.to_hdf(name,"ev_df")

In [22]:
t_df,p_df,pot_df = convert_root(root_filenames_CV)
ev_df = create_event_df(t_df,p_df)
write_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/CVSet/mcana_dfs.h5",
          t_df,p_df,pot_df,ev_df)

Processing 39991 files
	Processed 500 files. 10928 events processed.
	Processed 1000 files. 22040 events processed.
	Processed 1500 files. 32944 events processed.
	Processed 2000 files. 44097 events processed.
	Processed 2500 files. 55145 events processed.
	Processed 3000 files. 66415 events processed.
	Processed 3500 files. 77565 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/CVSet/Run3/AB/sampler_hist_6106.root, trees not found.
	Processed 4000 files. 88617 events processed.
	Processed 4500 files. 99795 events processed.
	Processed 5000 files. 110985 events processed.
	Processed 5500 files. 121751 events processed.
	Processed 6000 files. 132932 events processed.
	Processed 6500 files. 143947 events processed.
	Processed 7000 files. 154966 events processed.
	Processed 7500 files. 166007 events processed.
	Processed 8000 files. 177046 events processed.
	Processed 8500 files. 188162 events processed.
	Processed 9000 files. 199311 events processed.
	Processed 9500 fi

In [23]:
t_df,p_df,pot_df = convert_root(root_filenames_Set1)
ev_df = create_event_df(t_df,p_df)
write_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs.h5",
          t_df,p_df,pot_df,ev_df)

Processing 28537 files
	Processed 500 files. 8291 events processed.
	Processed 1000 files. 13640 events processed.
	Processed 1500 files. 19017 events processed.
	Processed 2000 files. 24375 events processed.
	Processed 2500 files. 29770 events processed.
	Processed 3000 files. 35127 events processed.
	Processed 3500 files. 40520 events processed.
	Processed 4000 files. 45720 events processed.
	Processed 4500 files. 51118 events processed.
	Processed 5000 files. 56442 events processed.
	Processed 5500 files. 61894 events processed.
	Processed 6000 files. 67304 events processed.
	Processed 6500 files. 72531 events processed.
	Processed 7000 files. 77882 events processed.
	Processed 7500 files. 83228 events processed.
	Processed 8000 files. 88671 events processed.
	Processed 8500 files. 94279 events processed.
	Processed 9000 files. 99562 events processed.
	Processed 9500 files. 104917 events processed.
	Processed 10000 files. 110309 events processed.
	Processed 10500 files. 119544 event

In [24]:
t_df,p_df,pot_df = convert_root(root_filenames_Set2)
ev_df = create_event_df(t_df,p_df)
write_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/mcana_dfs.h5",
          t_df,p_df,pot_df,ev_df)

Processing 48390 files
	Processed 500 files. 9045 events processed.
	Processed 1000 files. 18303 events processed.
	Processed 1500 files. 27356 events processed.
	Processed 2000 files. 36163 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/Run3/32044093_1706/sampler_hist_1706.root, trees not found.
	Processed 2500 files. 45546 events processed.
	Processed 3000 files. 54681 events processed.
	Processed 3500 files. 63953 events processed.
	Processed 4000 files. 73347 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/Run3/32044121_1183/sampler_hist_1183.root, trees not found.
	Processed 4500 files. 82819 events processed.
	Processed 5000 files. 92318 events processed.
	Processed 5500 files. 101552 events processed.
	Processed 6000 files. 110767 events processed.
	Processed 6500 files. 120243 events processed.
	Processed 7000 files. 129437 events processed.
	Processed 7500 files. 138944 events processed.
	Processed 8000 files. 148457 events pr

In [ ]:
t_df,p_df,pot_df = convert_root(root_filenames_Set3)
ev_df = create_event_df(t_df,p_df)
write_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/mcana_dfs.h5",
          t_df,p_df,pot_df,ev_df)

Processing 48505 files
	Processed 500 files. 8557 events processed.
	Processed 1000 files. 17345 events processed.
	Processed 1500 files. 26125 events processed.
	Processed 2000 files. 34915 events processed.
	Processed 2500 files. 43301 events processed.
	Processed 3000 files. 52078 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/Run3/31873900_6782/sampler_hist_6782.root, trees not found.
	Processed 3500 files. 60427 events processed.
	Processed 4000 files. 69069 events processed.
	Processed 4500 files. 77436 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/Run3/7265654_7040/sampler_hist_7040.root, trees not found.
	Processed 5000 files. 85995 events processed.
	Processed 5500 files. 94359 events processed.
	Processed 6000 files. 102968 events processed.
	Processed 6500 files. 111575 events processed.


In [ ]:
t_df,p_df,pot_df = convert_root(root_filenames_Set4)
ev_df = create_event_df(t_df,p_df)
write_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set4/mcana_dfs.h5",
          t_df,p_df,pot_df,ev_df)